In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box
from statmechlib.read_write import params_to_pickle

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
stats_file = 'stats_manyall' # 'stats_samples'
target_file = 'target_manyall' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [6]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)
    
with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [7]:
def downselect(stats_inp, pair_knots, edens_knots):
    
    # find idices of knots
    pair_index = find_index(pair_knots, stats_inp['hyperparams']['pair'])
    edens_index = find_index(edens_knots, stats_inp['hyperparams']['edens'])
    
    # create boolean arrays with select indices set to True
    p_ix = np.array([True if i in pair_index else False for i in range(len(stats_inp['hyperparams']['pair']))])
    m_ix = np.array([True if i in edens_index else False for i in range(len(stats_inp['hyperparams']['edens']))])

    stats_out = select_nodes(stats_inp, p_ix, m_ix)
    
    return stats_out

In [8]:
# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
# edens_knots = [ 2.22, 2.5,  2.6946925, 2.8663175, 3.1,  3.5,  4.5]

# pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525, 5.401695, 5.4604375]

pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
              3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
              4.8953, 5.18, 5.38, 5.58, 5.62]

#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.5]
#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.62]
edens_knots = [2.22, 2.5, 2.8663175000000001, 3.1, 3.5, 4.62]



In [9]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.1799999999999997, 5.3799999999999999, 5.5800000000000001, 5.6200000000000001], u'edens': [2.2200000000000002, 2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.6200000000000001]}
{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.1799999999999997, 5.3799999999999999, 5.5800000000000001, 5.6200000000000001], u'edens': [2.2200000000000002, 2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.6200000000000001]}


In [10]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [11]:
# for key in sorted(targets.keys()):
#     for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
#         n = len(xyz)
#         vol = np.linalg.det(box)
#         corr = float(n*n)/vol
#         ene.append([corr])

In [12]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.1
targets['relax']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [13]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [14]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('md', 1.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('relax', 0.0),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('eos', 0.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('liq_5000K', 1.0),
 ('eos_bcc', 0.1),
 ('bain', 0.0)]

In [15]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])

# start_params.append([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
#          3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
#         -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
#         -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
#          7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
#         -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
#          1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
#          2.18327852e+00,   6.63152141e-02, 0.0])

# start_params.append([ -2.95016477e+00,   3.78177969e-04,  -5.21095611e+02,
#          3.09845244e+02,  -4.69222440e+02,   3.95186037e+02,
#         -1.11630659e+02,   5.36915127e+00,   1.52984702e+01,
#         -1.54915170e+01,   7.53107481e+00,  -2.98591742e+00,
#          7.55346307e-01,  -1.00678488e+00,   1.96634582e+00,
#         -8.71560927e-01,  -1.41820832e-01,  -1.46508000e+00,
#          1.42936365e+00,   7.23805795e+01,   0.0, 9.18071460e-01,
#          2.04259992e+00,   6.28575567e-02,   7.62804667e-03])

# start_params.append([ -3.27882058e+00,  -9.77681860e-04,  -5.95486537e+02,
#          3.42401535e+02,  -4.15497479e+02,   3.71316746e+02,
#         -1.07274610e+02,  -2.93282898e+00,   1.51298572e+01,
#         -1.37454674e+01,   8.16724633e+00,  -3.16850352e+00,
#          8.74607042e-01,  -1.07878768e+00,   1.97939820e+00,
#         -8.26149646e-01,  -1.62437297e-01,  -1.45694910e+00,
#          1.42153527e+00,   1.29203135e+02,  -9.25284676e+00,
#          2.63967417e+00,   2.06410900e+00,   6.03258240e-02*2])

# start_params.append([ -3.56709315e+00,  -2.97358686e-03,  -4.37632666e+02,
#          3.35095381e+02,  -2.95083648e+02,   2.94263473e+02,
#         -8.57731610e+01,  -1.62036228e+01,   1.30838141e+01,
#         -6.46551376e+00,   8.01826587e+00,  -3.58786095e+00,
#          1.25735987e+00,  -1.21627097e+00,   2.04033545e+00,
#         -8.04702913e-01,  -1.92666480e-01,  -1.44083371e+00,
#          1.41816098e+00,   1.70360405e+02,  -1.89070431e+01,
#          7.47139804e+00,   1.20540194e+00,   1.34588181e-01, 0.0])

# start_params.append([ -3.89054208e+00,  -2.99469508e-03,  -7.60325091e+02,
#          2.25922421e+02,   4.31682139e+01,  -2.00879587e+01,
#          3.15977299e+01,  -1.41216383e+01,   1.40729549e+01,
#         -2.29741069e-01,   5.66020784e-01,   9.50813370e-01,
#         -7.07877506e-01,  -6.33885976e-01,   2.11514130e+00,
#         -1.07084535e+00,  -1.99471358e-01,  -1.53976066e+00,
#          1.54749579e+00,   1.94668273e+02,  -1.93218649e+00,
#          4.87330211e+00,  -4.95019886e-01,   1.54561460e-01])

# start_params.append([ -3.01252922e+00,   3.34774026e-03,  -3.63245948e+02,
#          7.08473344e+01,   5.18269140e+00,  -4.21852665e+01,
#          5.03986439e+01,  -3.49299323e+01,   9.39050841e+00,
#          7.10916965e+00,   2.53141800e+00,  -1.27714028e-01,
#         -8.13152443e-01,  -5.45294030e-01,   2.19662624e+00,
#         -1.03021233e+00,  -2.37376783e-01,  -1.54945600e+00,
#          1.51410989e+00,   8.93873263e+01,  -2.44415218e+01,
#          1.44237601e+01,  -2.48517486e-01,   1.57735701e-01,
#         0.0])

# start_params.append([ -2.76762846e+00,   1.89330096e-03,  -3.78969507e+02,
#          4.97375946e+01,   5.61080362e+01,  -8.13991417e+01,
#          6.18213043e+01,  -3.36881958e+01,   9.64679581e+00,
#          6.37021438e+00,   2.56205745e+00,  -6.89271288e-02,
#         -8.28256696e-01,  -5.90663693e-01,   2.19738636e+00,
#         -9.78519868e-01,  -2.34054246e-01,  -1.56404102e+00,
#          1.50934562e+00,   1.25199350e+02,  -2.58933860e+01,
#          1.60206690e+01,  -2.46344691e-01,   1.98323959e-01,
#         0.0])

# start_params.append([ -2.96336623e+00,   9.35841772e-04,  -4.69405427e+02,
#          6.56489326e+01,   6.59098372e+01,  -6.95558862e+01,
#          5.05321941e+01,  -2.68901225e+01,   6.12983979e+00,
#          8.40257564e+00,   2.03599185e+00,   1.44655457e-01,
#         -8.07626370e-01,  -6.77900254e-01,   2.19493850e+00,
#         -9.02854774e-01,  -2.35346587e-01,  -1.56947636e+00,
#          1.49891405e+00,   1.54692423e+02,  -1.84128239e+01,
#          1.40416342e+01,  -4.74336808e-01,   2.07514499e-01])

# start_params.append([ -2.83157185e+00,   8.39115358e-04,  -4.21330968e+02,
#          5.62354559e+01,   6.55868693e+01,  -7.39097694e+01,
#          5.32753168e+01,  -2.90768080e+01,   9.73480693e+00,
#          5.86938247e+00,   2.40344261e+00,   1.33807130e-01,
#         -7.34188649e-01,  -8.53691403e-01,   2.20105517e+00,
#         -7.61586871e-01,  -2.42532670e-01,  -1.58457873e+00,
#          1.47655687e+00,   0.0, 1.48755630e+02,  0.0, -1.98109911e+01,
#          1.44803287e+01,  -1.17489342e-01,   2.06651900e-01])

# start_params.append([ -3.91274686e+00,  -5.02814282e-03,  -3.60607144e+02,
#          3.08180837e+02,  -2.45731624e+02,   2.57711860e+02,
#         -7.57560376e+01,  -3.26396342e+01,   1.25979333e+01,
#         -3.17982854e+00,   7.88108320e+00,  -3.46868286e+00,
#          1.29424331e+00,  -1.31629778e+00,   2.05422384e+00,
#         -8.33747039e-01,  -1.75108086e-01,  -1.43404985e+00,
#          1.45403961e+00,   1.43868098e+02,  -2.72551797e+01,
#          8.05592823e+00,   1.03649030e+00,   1.04857599e-01,
#         -4.20237149e+01])

# start_params.append([ -3.34519702e+00,   7.66822271e-04,  -4.03553412e+02,
#          1.55282185e+01,   2.90516401e+01,  -6.23087513e+01,
#          8.48184009e+01, -3.48806112e+01,   9.13550274e+00,
#          5.96096071e+00,   2.54062374e+00,   2.39873080e-01,
#         -7.19239652e-01,  -1.04267291e+00,   2.29366373e+00,
#         -7.05206149e-01,  -2.72279799e-01,  -1.55954882e+00,
#          1.45964251e+00,  -2.01431222e+02,   3.24297105e+01,
#          3.81664135e+01,  -2.18948938e+01,   1.02484895e+01,
#          2.20300623e-01,   1.13874983e-01])

start_params.append([ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
         1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
         1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
        -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
        -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
        -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
         1.27167940e+00,   -1.05537460e-02,   1.06186365e+02,   2.86689290e+01,
        -2.47374985e+00,  -1.24252596e+00,   1.65373019e-01])

multi_pars = start_params
print(multi_pars)

[[-4.65804603, -0.00756457588, -168.77752, 156.052349, 14.2649908, -4.55941009, 19.761334, 48.4204936, 6.4946677, -6.62233324, -2.16547494, 1.30932417, -0.83328148, -0.832333597, 2.09417158, -0.619566076, -0.0953112547, -1.49756931, 1.2716794, -0.010553746, 106.186365, 28.668929, -2.47374985, -1.24252596, 0.165373019]]


In [16]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [17]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-4.65804603, -0.00756457588, -168.77752, 156.052349, 14.2649908, -4.55941009, 19.761334, 48.4204936, 6.4946677, -6.62233324, -2.16547494, 1.30932417, -0.83328148, -0.832333597, 2.09417158, -0.619566076, -0.0953112547, -1.49756931, 1.2716794, -0.010553746, 106.186365, 28.668929, -2.47374985, -1.24252596, 0.165373019])
loss 0.25302141558 0.25302141558 0.0
('Best params:', [(0.25302141558037061, [-4.65804603, -0.00756457588, -168.77752, 156.052349, 14.2649908, -4.55941009, 19.761334, 48.4204936, 6.4946677, -6.62233324, -2.16547494, 1.30932417, -0.83328148, -0.832333597, 2.09417158, -0.619566076, -0.0953112547, -1.49756931, 1.2716794, -0.010553746, 106.186365, 28.668929, -2.47374985, -1.24252596, 0.165373019])])
('ieam:', 0)
loss 0.25302141558 0.25302141558 0.0
loss 0.259300983156 0.259300983156 0.0
loss 0.253422141595 0.253422141595 0.0
loss 0.253020999647 0.253020999647 0.0
loss 0.253085349532 0.253085349532 0.0
loss 0.253046186074 0.253046186074 0.0
loss 0.253009281

loss 0.00645512641092 0.00645512641092 0.0
loss 0.00609732877707 0.00609732877707 0.0
loss 0.00587851532846 0.00587851532846 0.0
loss 0.0081819456745 0.0081819456745 0.0
loss 0.0058214415298 0.0058214415298 0.0
loss 0.00729364657496 0.00729364657496 0.0
loss 0.00680026965933 0.00680026965933 0.0
loss 0.00697877836174 0.00697877836174 0.0
loss 0.00633435715533 0.00633435715533 0.0
loss 0.00928878775196 0.00928878775196 0.0
loss 0.00543239898357 0.00543239898357 0.0
loss 0.00920842218311 0.00920842218311 0.0
loss 0.00545651388761 0.00545651388761 0.0
loss 0.00518071361412 0.00518071361412 0.0
loss 0.00466124646468 0.00466124646468 0.0
loss 0.00428402618927 0.00428402618927 0.0
loss 0.00651837888668 0.00651837888668 0.0
loss 0.00402936456825 0.00402936456825 0.0
loss 0.0074489004993 0.0074489004993 0.0
loss 0.00519361935011 0.00519361935011 0.0
loss 0.00426001969428 0.00426001969428 0.0
loss 0.00418856460975 0.00418856460975 0.0
loss 0.00419147361908 0.00419147361908 0.0
loss 0.0038365410

Process PoolWorker-3:
Process PoolWorker-2:
Process PoolWorker-4:
Process PoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    se

KeyboardInterrupt: 

In [19]:
# params_output = [ -4.72546533e+00,  -6.97068121e-03,  -3.51290250e+02,
#          1.63175272e+02,   9.43598046e+00,  -3.25684355e+00,
#          2.30446047e+01,   4.06678582e+01,   7.56509137e+00,
#         -7.51663487e+00,  -1.59146529e+00,   1.14098894e+00,
#         -8.25274736e-01,  -8.09954101e-01,   2.09984210e+00,
#         -6.52067950e-01,  -8.56530962e-02,  -1.49919134e+00,
#          1.27499075e+00,   1.07226915e+02,   2.36918896e+01,
#         -1.29592031e+00,  -1.49967198e+00,   2.14664790e-01]
# params_output = [[ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01]]

# params_output = [[ -4.96571997e+00,  -9.70695776e-03,  -2.69597001e+03,
#          1.06172942e+02,  -9.45431278e+01,   3.88799435e+01,
#          8.37612190e+00,   5.83170234e+01,   7.58236573e+00,
#         -7.89909608e+00,  -2.26521289e+00,   1.17286424e+00,
#         -8.71880389e-01,  -8.53217217e-01,   2.11019782e+00,
#         -6.07750055e-01,  -1.50403641e-01,  -1.45924396e+00,
#          1.28914484e+00,  -1.33764316e+03,   2.53825479e+01,
#          2.81342191e+01,  -1.13926920e+00,  -1.62929262e+00,
#          1.66534929e-01]]

params_output = [
    -4.96571997, -0.00970695776,
    -4.23764410e+03,   9.82966281e+01,  -6.09610499e+01,
        -1.41310061e+01,   4.09003482e+01,   4.59147811e+01,
         1.69148405e+01,  -1.08920404e+01,  -2.15512880e+00,
         1.19928506e+00,  -9.64477377e-01,  -5.06943698e-01,
         1.54791512e+00,  -6.82398422e-01,   1.58195979e-02,
        -5.69663710e-01,   5.77918066e-01,
    -1337.64316,25.3825479,28.1342191,
    -1.1392692,-1.62929262,0.166534929 
]

print(params_output)

[[-4.96571997, -0.00970695776, -2695.97001, 106.172942, -94.5431278, 38.8799435, 8.3761219, 58.3170234, 7.58236573, -7.89909608, -2.26521289, 1.17286424, -0.871880389, -0.853217217, 2.11019782, -0.607750055, -0.150403641, -1.45924396, 1.28914484, -1337.64316, 25.3825479, 28.1342191, -1.1392692, -1.62929262, 0.166534929]]


In [20]:
# convert lists of optimal parameters to dictionaries
params_dicts = []
for params in params_output:
    params_dicts.append(to_param_dict(params, stats['hyperparams']))

In [21]:
params_dicts

[{'hyperparams': {u'edens': [2.2200000000000002,
    2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.6200000000000001],
   u'pair': [2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [-1337.64316,
    25.3825479,
    28.1342191,
    -1.1392692,
    -1.62929262,
    0.166534929],
   'embed': [-4.96571997, -0.00970695776],
   'lrcorr': [],
   'pair': [-2695.97001,
    106.172942,
    -94.5431278,
    38.8799435,
    8.3761219,
    58.3170234,
    7.58236573,
    -7.89909608,
    -2.26521289,
    1.17286424,
    -0.871880389,
    -0.853217217,
    2.11019782,
    -0.607750055,
    -0.150403641,
    -1.45924396,
    1.289

In [22]:
file_name = os.path.join(working, "params_out.pickle")

for params in params_dicts:
    params_to_pickle(params, file_name)

In [23]:
with open(os.path.join(working, 'params_out' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [24]:
retreived_params

[{'hyperparams': {u'edens': [2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.5],
   u'pair': [2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [106.853821,
    27.4234479,
    -1.26300179,
    -1.66378666,
    0.224848394],
   'embed': [-4.65804603, -0.00756457588],
   'lrcorr': [],
   'pair': [-168.77752,
    156.052349,
    14.2649908,
    -4.55941009,
    19.761334,
    48.4204936,
    6.4946677,
    -6.62233324,
    -2.16547494,
    1.30932417,
    -0.83328148,
    -0.832333597,
    2.09417158,
    -0.619566076,
    -0.0953112547,
    -1.49756931,
    1.2716794]}},
 {'hyperparams': {u'edens': [2.2200000000000002,